<pre>Name      :   Chaitanya Machhindra Nawale
Roll No.  :   BECOB215</pre>

##Import dataset

In [31]:
from sklearn.datasets import fetch_20newsgroups

####Select categories

*   List item
*   List item



In [32]:
categories =['comp.graphics', 'sci.med','rec.sport.baseball','rec.sport.hockey']

###Load training and testing data
####train_data and test_data store data as a dict

In [33]:
train_data = fetch_20newsgroups(subset='train', categories=categories,shuffle=True)
test_data = fetch_20newsgroups(subset='test',categories=categories,shuffle=True)

In [34]:
train_data.keys()


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [35]:
#categories
train_data['target_names']

['comp.graphics', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.med']

In [36]:
print((train_data.data[0]))

From: tedward@cs.cornell.edu (Edward [Ted] Fischer)
Subject: Re: Ind. Source Picks Baerga Over Alomar: Case Closed 
Organization: Cornell Univ. CS Dept, Ithaca NY 14853
Distribution: na
Lines: 24

In article <C5L6Dn.4uB@andy.bgsu.edu> klopfens@andy.bgsu.edu (Bruce Klopfenstein) writes:
>fester@island.COM (Mike Fester) writes:
>> 
>> I'd say you could make a good for them being about equal right now. T&P
>> rated Baerga higher, actually.
>
>Finally, an objective source.  Alomar's a great player, but so is Baerga.
>Nice to see the objective source cited rather than "my dad's bigger than
>your dad" posts.

I know.  You have this fucked up idea that anybody who prefers Alomar
to Baerga must be a Jay-Lover and Indian-Hater.  Sorry, you got that
one wrong!  I hate the Jays and don't care one way or the other about
the Indians.  But objectively, Alomar had the better offensive year
last year, so I have to pick him.

You admit T&P as a reliable(?), objective source?  Then you will note
that th

##Removing stopwords and stemming

In [37]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwrds = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

for i in range(len(train_data.data)):
  text = train_data.data[i]
  text = re.sub('[^a-zA-Z]' , " " ,text)
  text = text.lower()
  text = text.split()
  text =[ps.stem(x) for x in text if not x in set(stopwrds)]
  text =" ".join(text)
  train_data.data[i] = text

for i in range(len(test_data.data)):
  text = test_data.data[i]
  text = re.sub('[^a-zA-Z]' , " " ,text)
  text = text.lower()
  text = text.split()
  text =[ps.stem(x) for x in text if not x in set(stopwrds)]
  text =" ".join(text)
  test_data.data[i] = text
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Word Count - CountVectorizer

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train = cv.fit_transform(train_data.data)

print((cv.get_feature_names()))
x_train.shape

['aa', 'aaa', 'aaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg', 'aaai', 'aad', 'aalborg', 'aamrl', 'aanerud', 'aangeboden', 'aantal', 'aao', 'aaoepp', 'aaplay', 'aardvark', 'aargh', 'aarhu', 'aarnet', 'aaron', 'aaronson', 'aau', 'ab', 'abacu', 'abad', 'abandon', 'abandond', 'abat', 'abber', 'abbot', 'abbott', 'abbrev', 'abbrevi', 'abc', 'abcd', 'abdomen', 'abdomin', 'abduct', 'abe', 'abeit', 'abeka', 'abel', 'aberdeen', 'aberr', 'abhin', 'abi', 'abil', 'abild', 'abildskov', 'abington', 'abl', 'ablaz', 'abner', 'abnorm', 'abo', 'abort', 'abott', 'abotu', 'abp', 'abpsoft', 'abras', 'abrash', 'abraxi', 'abreast', 'abscess', 'absenc', 'absent', 'absol', 'absolut', 'absorb', 'absorbt', 'absorpt', 'abstact', 'abstract', 'abstractsoft', 'absurd', 'absurdum', 'abund', 'abus', 'abysm', 'abyss', 'ac', 'acad', 'academ', 'academi', 'academia', 'acadia', 'acadiau', 'acadien', 'acap', 'acc', 'acceler', 'accentu', 'accept', 'acces', 'access', 'accessori', 'acc

(2375, 22452)

## TFIDF


In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
x_train_tfidf = tfidf.fit_transform(x_train)
x_train_tfidf.shape

(2375, 22452)

##MultinomialNB classifier

In [40]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_tfidf , train_data.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [41]:
x_test = cv.transform(test_data.data)
x_test_tfidf = tfidf.transform(x_test)
predicted = model.predict(x_test_tfidf)

In [42]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("accuracy score= " + str( accuracy_score(test_data.target , predicted)) +"\n\n")
print(metrics.classification_report(test_data.target , predicted ,target_names=test_data.target_names))

accuracy score= 0.9506641366223909


                    precision    recall  f1-score   support

     comp.graphics       0.96      0.94      0.95       389
rec.sport.baseball       0.96      0.94      0.95       397
  rec.sport.hockey       0.94      0.99      0.96       399
           sci.med       0.95      0.93      0.94       396

          accuracy                           0.95      1581
         macro avg       0.95      0.95      0.95      1581
      weighted avg       0.95      0.95      0.95      1581

